In [1]:
import json
import os
from utils.data_dirs import data_dirs
from utils.filter_road import filter_road

In [2]:
d = data_dirs()
print(d.raw_road)
print(d.processed_road)
print(d.raw_map)
print(d.processed_map)

/work/apperception/data/raw/road-network
/data/apperception-data/processed/road-network
/work/apperception/data/raw/nuScenes/Map-expansion
/data/apperception-data/processed/nuscenes/Map-expansion


In [3]:
raw_expansion = os.path.join(d.raw_map, 'expansion')
processed_expansion = os.path.join(d.processed_map, 'expansion')

In [4]:
raw_expansion

'/work/apperception/data/raw/nuScenes/Map-expansion/expansion'

In [5]:
processed_expansion

'/data/apperception-data/processed/nuscenes/Map-expansion/expansion'

In [6]:
int('f', 16)

15

In [7]:
EXAMPLE_TOKEN = '0073298b-b2f4-4f89-97cd-4241a1599831'

def is_hex(h: 'str'):
    try:
        int(h, 16)
        return True
    except:
        return False

def is_token(token) -> "bool":
    if not isinstance(token, str):
        return False

    chunks = token.split('-')
    if len(chunks) != 5:
        return False
    
    if [len(c) for c in chunks] != [8, 4, 4, 4, 12]:
        return False
    
    return all(map(is_hex, chunks))

In [8]:
import itertools

[*itertools.chain([1,2], [4], itertools.chain(*(x for x in [[1], [2]])), map(lambda x: x, [2]))]

[1, 2, 4, 1, 2, 2]

In [9]:
import itertools

def extract_all_tokens(o):
    if isinstance(o, dict):
        return itertools.chain(itertools.chain(*map(extract_all_tokens, o.values())), itertools.chain(*map(extract_all_tokens, o.keys())))
    elif isinstance(o, list) or isinstance(o, tuple):
        return itertools.chain(*map(extract_all_tokens, o))
    elif is_token(o):
        return [o]
    else:
        return []

def extract_all_non_tokens(o):
    if isinstance(o, dict):
        return itertools.chain(*map(extract_all_non_tokens, o.values()))
    elif isinstance(o, list):
        return itertools.chain(*map(extract_all_non_tokens, o))
    elif is_token(o):
        return []
    else:
        if not (isinstance(o, int) or isinstance(o, float)):
            return [o]
        return []

In [10]:
all_non_tokens = set()
tokens = dict()
for e in os.listdir(raw_expansion):
    with open(os.path.join(raw_expansion, e), "r") as f:
        e_json = json.load(f)
    key = e.split('.')[0]
    tokens[key] = {*extract_all_tokens(e_json)}
    all_non_tokens.update(extract_all_non_tokens(e_json))

print(all_non_tokens)
print(tokens.keys())

{'', 'NIL', 'UP', 'SINGLE_ZIGZAG_WHITE', 'SINGLE_SOLID_WHITE', 'YIELD', 'CAR', 'STOP_SIGN', 'LSR', '1.3', 'DOUBLE_DASHED_WHITE', 'VERTICAL', 'YELLOW', 'TRAFFIC_LIGHT', 'LEFT', 'RIGHT', 'TURN_STOP', 'GREEN', 'SINGLE_SOLID_YELLOW', 'RED', 'RSL', 'HORIZONTAL', 'RSR', 'PED_CROSSING', 'DOUBLE_SOLID_WHITE', 'CIRCLE', 'LSL', None}
dict_keys(['boston-seaport', 'singapore-queenstown', 'singapore-onenorth', 'singapore-hollandvillage'])


In [11]:
_tokens = []
for e in os.listdir(raw_expansion):
    with open(os.path.join(raw_expansion, e), "r") as f:
        e_json = json.load(f)
    
#     tokens: "Set[str]" = set()
        
#     for k, value in e_json.items():
#         if type(value) == list:
#             tokens.update({v["token"] for v in value if type(v) == dict})
#         elif type(value) == dict:
#             tokens.update(value.keys())
    tokens = {*extract_all_tokens(e_json)}
    
    if not os.path.exists(processed_expansion):
        os.makedirs(processed_expansion)
    
    with open(os.path.join(processed_expansion, e[:-len(".json")] + "-tokens.json"), "w") as f:
        json.dump([*tokens], f)
    
    _tokens.append(tokens)

for t1 in _tokens:
    for t2 in _tokens:
        if t1 == t2: continue
        print(t1 & t2)

set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()


In [12]:
for tokens_f in os.listdir(processed_expansion):
    with open(os.path.join(processed_expansion, tokens_f), "r") as f:
        tokens = {*json.load(f)}
    
    region = tokens_f[:-len("-tokens.json")]
    print(region)
    filtered = filter_road(d.raw_road, tokens)
    for k, v in filtered.items():
        print(k, len(v))
        base = os.path.join(d.processed_road, region)
        if not os.path.exists(base):
            os.makedirs(base)
        with open(os.path.join(base, k + ".json"), "w") as f:
            json.dump(v, f, indent=1)
    

singapore-hollandvillage
Filtering intersection
Filtering lane_LaneSec
Filtering lane
Filtering laneGroup_Lane
Filtering laneGroup_opposite
Filtering laneGroup
Filtering laneSection
Filtering polygon
Filtering road_laneGroup
Filtering road_roadSec
Filtering road
Filtering roadSec_laneSec
Filtering roadSection
Filtering segment
intersection 167
lane_LaneSec 0
lane 0
laneGroup_Lane 0
laneGroup_opposite 0
laneGroup 0
laneSection 0
polygon 334
road_laneGroup 0
road_roadSec 0
road 167
roadSec_laneSec 0
roadSection 0
segment 167
singapore-onenorth
Filtering intersection
Filtering lane_LaneSec
Filtering lane
Filtering laneGroup_Lane
Filtering laneGroup_opposite
Filtering laneGroup
Filtering laneSection
Filtering polygon
Filtering road_laneGroup
Filtering road_roadSec
Filtering road
Filtering roadSec_laneSec
Filtering roadSection
Filtering segment
intersection 293
lane_LaneSec 933
lane 933
laneGroup_Lane 933
laneGroup_opposite 300
laneGroup 632
laneSection 933
polygon 4045
road_laneGroup 629
r

In [13]:
base_old = os.path.join(d.processed_road, "boston-old")
base_seaport = os.path.join(d.processed_road, "boston-seaport")

def remove_heading(d):
    if 'heading' in d:
        del d['heading']
    return d

for e in os.listdir(base_old):
    with open(os.path.join(base_old, e), "r") as f:
        old = json.load(f)
    with open(os.path.join(base_seaport, e), "r") as f:
        seaport = json.load(f)
    
    print(e)
    old_set = set((json.dumps(o, sort_keys=True) for o in map(remove_heading, old)))
    seaport_set = set((json.dumps(o, sort_keys=True) for o in map(remove_heading, seaport)))
    print('boston-old is subset:', len(old_set - seaport_set) == 0)
    print()
    

laneGroup.json
boston-old is subset: True

roadSec_laneSec.json
boston-old is subset: True

road.json
boston-old is subset: True

segment.json
boston-old is subset: True

road_roadSec.json
boston-old is subset: True

lane.json
boston-old is subset: True

intersection.json
boston-old is subset: True

laneSection.json
boston-old is subset: True

laneGroup_opposite.json
boston-old is subset: True

polygon.json
boston-old is subset: True

lane_LaneSec.json
boston-old is subset: True

laneGroup_Lane.json
boston-old is subset: True

road_laneGroup.json
boston-old is subset: True

roadSection.json
boston-old is subset: True



In [14]:
with open(os.path.join(base_old, 'polygon.json'), "r") as f:
    old = json.load(f)
with open(os.path.join(base_seaport, 'polygon.json'), "r") as f:
    seaport = json.load(f)
# old[0]
print([o for o in old if o['id'] == "971af7a8-855a-474a-8715-f6b8016e78ff"])
print([o for o in seaport if o['id'] == "971af7a8-855a-474a-8715-f6b8016e78ff"])

[{'id': '971af7a8-855a-474a-8715-f6b8016e78ff', 'polygon': 'POLYGON ((2525.6528905288387250 870.0673772124993093, 2532.6247537459998966 872.5188277609444185, 2532.3638174254715523 871.7643520778366337, 2532.0731055887108596 869.9739163944096845, 2532.1660582513122790 863.9078996779289810, 2532.2265815159626072 862.7908341711502089, 2532.4372961932372164 858.7061014861466219, 2527.9970330620476489 858.0422771163575817, 2527.9222186058777879 862.6161804021471653, 2527.3011173659033375 866.2497474613342092, 2525.6528905288387250 870.0673772124993093))'}]
[{'id': '971af7a8-855a-474a-8715-f6b8016e78ff', 'polygon': 'POLYGON ((2525.6528905288387250 870.0673772124993093, 2532.6247537459998966 872.5188277609444185, 2532.3638174254715523 871.7643520778366337, 2532.0731055887108596 869.9739163944096845, 2532.1660582513122790 863.9078996779289810, 2532.2265815159626072 862.7908341711502089, 2532.4372961932372164 858.7061014861466219, 2527.9970330620476489 858.0422771163575817, 2527.922218605877787

In [15]:
with open(os.path.join(base_old, 'segment.json'), "r") as f:
    old = json.load(f)
with open(os.path.join(base_seaport, 'segment.json'), 'r') as f:
    seaport = json.load(f)
print(json.dumps([o for o in old if o['polygonId'] == "971af7a8-855a-474a-8715-f6b8016e78ff"], indent=2))
print(json.dumps([o for o in seaport if o['polygonId'] == "971af7a8-855a-474a-8715-f6b8016e78ff"], indent=2))

[
  {
    "start": "POINT (2532.4372961932372164 858.7061014861466219)",
    "end": "POINT (2532.2265815159626072 862.7908341711502089)",
    "heading": 0.051540230404979015,
    "polygonId": "971af7a8-855a-474a-8715-f6b8016e78ff"
  },
  {
    "start": "POINT (2532.2265815159626072 862.7908341711502089)",
    "end": "POINT (2532.1660582513122790 863.9078996779289810)",
    "heading": 0.054127663628520395,
    "polygonId": "971af7a8-855a-474a-8715-f6b8016e78ff"
  },
  {
    "start": "POINT (2532.1660582513122790 863.9078996779289810)",
    "end": "POINT (2532.0731055887108596 869.9739163944096845)",
    "heading": 0.015322309938731404,
    "polygonId": "971af7a8-855a-474a-8715-f6b8016e78ff"
  },
  {
    "start": "POINT (2532.0731055887108596 869.9739163944096845)",
    "end": "POINT (2532.3638174254715523 871.7643520778366337)",
    "heading": -0.1609645906098347,
    "polygonId": "971af7a8-855a-474a-8715-f6b8016e78ff"
  },
  {
    "start": "POINT (2532.3638174254715523 871.764352077836